# Renta Nacional

In [1]:
import pandas as pd
df_sc = pd.read_excel("BBDD/SC_COTIZACIONES.xlsx")
print(df_sc.head())

  FECHA_COTIZACION  NUMERO_SOLICITUD  NUMERO_IDENTIFICACION  NUMERO_SECUENCIA  \
0       2024-01-02         125720401              188965131                 1   
1       2024-01-02         125720401              188965150                 1   
2       2024-01-02         125720401              188965169                 1   
3       2024-01-02         125720401              188965188                 1   
4       2024-01-02         125747401              188975930                 1   

  TIPO_RENTA  MESES_DIFERIDOS MODALIDAD_RENTA  MESES_GARANTIZADOS  \
0          I                0               S                   0   
1          I                0               G                 120   
2          I                0               G                 192   
3          I                0               G                 180   
4          I                0               S                   0   

   MESES_AUMENTO_TEMPORAL  PORCENTAJE_AUMENTO_TEMPORAL  ...  PRIMA_UNICA  \
0                     

Cargo el otro df para tratar de cruzarlos.

In [2]:
import pandas as pd
df = pd.read_excel("BBDD/COTIZACIONES.xlsx")
print(df.head())

  FECHA_COTIZACION TIPO_PENSION TIPO_COTIZACION  COTIZANTE  COTIZACION  \
0       2024-01-02            S               E  421008240           1   
1       2024-01-02            S               E  421008240           2   
2       2024-01-02            S               E  421008240           3   
3       2024-01-02            S               E  421008240           4   
4       2024-01-02           VA               E  421005868          11   

  TIPO_INTERMEDIARIO RUT_CORREDOR TIPO_RENTA  MESES_DIFERIDOS MODALIDAD_RENTA  \
0                  S   11370691-0          I                0               S   
1                  S   11370691-0          I                0               G   
2                  S   11370691-0          I                0               G   
3                  S   11370691-0          I                0               G   
4                  S   13982859-3          I                0               G   

   ...        VAN  TASA_VAN  DURATION  DIAS_VALIDEZ  INVALIDA  TIPO_

Vemos si hay coincidencias.

In [20]:
print(f"Cotizantes: {len(df)}")
print(f"Cotizantes SC: {len(df_sc)}")

Cotizantes: 971410
Cotizantes SC: 962929


In [19]:
cotizantes_comunes = pd.merge(df, df_sc, on=['COTIZANTE', 'COTIZACION'], how='inner')
print(f"Cotizantes comunes: {len(cotizantes_comunes)}")

Cotizantes comunes: 962929


In [ ]:
cotizantes_comunes[(cotizantes_comunes['COTIZANTE'] == 421007881) & (cotizantes_comunes['COTIZACION'] == 1)]

,FECHA_COTIZACION_x,TIPO_PENSION,TIPO_COTIZACION,COTIZANTE,COTIZACION,TIPO_INTERMEDIARIO,RUT_CORREDOR,TIPO_RENTA_x,MESES_DIFERIDOS_x,MODALIDAD_RENTA_x,...,PRIMA_UNICA_y,RETENCION_AFP_y,RETIRO_EXCEDENTES_y,INVALIDA_y,TIPO_ERROR_y,POSICION_RELATIVA,SEGMENTO,NOMBRE_SEGMENTO,TIT_PROPUESTA,TIT_MAXIMA_y
0,2024-01-02,V,A,421007881,1,NaN,NaN,I,0,S,...,730.59,0.0,0.0,0,0,2,72,V 700-800 UF,3.26,3.28


In [22]:
df[(df['COTIZANTE'] == 421007881) & (df['COTIZACION'] == 1)]

,FECHA_COTIZACION,TIPO_PENSION,TIPO_COTIZACION,COTIZANTE,COTIZACION,TIPO_INTERMEDIARIO,RUT_CORREDOR,TIPO_RENTA,MESES_DIFERIDOS,MODALIDAD_RENTA,...,VAN,TASA_VAN,DURATION,DIAS_VALIDEZ,INVALIDA,TIPO_ERROR,EMITIDA,FECHA_RESPUESTA,ACEPTADA,FECHA_ACEPTACION
5,2024-01-02,V,A,421007881,1,NaN,NaN,I,0,S,...,33.49905,3.0,113.498,12,0.0,0.0,0,NaT,0,NaT


In [23]:
df_sc[(df_sc['COTIZANTE'] == 421007881) & (df_sc['COTIZACION'] == 1)]

,FECHA_COTIZACION,NUMERO_SOLICITUD,NUMERO_IDENTIFICACION,NUMERO_SECUENCIA,TIPO_RENTA,MESES_DIFERIDOS,MODALIDAD_RENTA,MESES_GARANTIZADOS,MESES_AUMENTO_TEMPORAL,PORCENTAJE_AUMENTO_TEMPORAL,...,PRIMA_UNICA,RETENCION_AFP,RETIRO_EXCEDENTES,INVALIDA,TIPO_ERROR,POSICION_RELATIVA,SEGMENTO,NOMBRE_SEGMENTO,TIT_PROPUESTA,TIT_MAXIMA
0,2024-01-02,125720401,188965131,1,I,0,S,0,0,0,...,730.59,0.0,0.0,0,0,2,72,V 700-800 UF,3.26,3.28


Realizamos el modelo.

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

# --- Paso 1: Preparación de datos ---

# 1.1 Muestra de 10.000 filas
df_sample = df_sc #.sample(n=100000, random_state=42).copy()

# 1.2 Conversión de columnas numéricas con comas
cols_to_convert = [
    "COTIZACION", "RENTA", "PRIMA_UNICA", 
    "RETENCION_AFP", "RETIRO_EXCEDENTES", 
    "TIT_PROPUESTA", "TIT_MAXIMA", 
    "PORCENTAJE_COMISION"
]

for col in cols_to_convert:
    df_sample[col] = df_sample[col].astype(str).str.replace(',', '.', regex=False).astype(float)

# 1.3 Codificación de variables categóricas
categorical_cols = ["TIPO_RENTA", "MODALIDAD_RENTA", "SEGMENTO", "NOMBRE_SEGMENTO"]
df_sample[categorical_cols] = df_sample[categorical_cols].astype("category")

# One-hot encoding
df_encoded = pd.get_dummies(df_sample, columns=categorical_cols)

# 1.4 Features (X) y etiqueta (y)
X = df_encoded.drop(columns=["POSICION_RELATIVA", "FECHA_COTIZACION"])
y = df_sample["POSICION_RELATIVA"].astype(int)

# Guardar índice original antes de imputar
X["index_original"] = df_sample.index

# Imputación de NaN con la media
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X.drop(columns=["index_original"])), columns=X.columns[:-1])
X_imputed["index_original"] = X["index_original"].values

# --- Paso 1.5: Filtrar clases con al menos 2 observaciones ---
conteo_clases = y.value_counts()
clases_validas = conteo_clases[conteo_clases >= 2].index

# Filtrar X e y con las clases válidas
filtro_validas = y.isin(clases_validas)
X_imputed = X_imputed[filtro_validas].reset_index(drop=True)
y = y[filtro_validas].reset_index(drop=True)


# --- Paso 2: Modelo de Random Forest ---

# 2.1 División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed.drop(columns=["index_original"]),
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

index_test = X_imputed.loc[X_test.index, "index_original"]

# 2.2 Entrenamiento
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    class_weight="balanced"
)
clf.fit(X_train, y_train)

# 2.3 Evaluación
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# --- Paso 3: Resultados y exportación ---

# 3.1 Probabilidades por clase
y_prob = clf.predict_proba(X_test)
prob_df = pd.DataFrame(y_prob, columns=[f"proba_pos_{cls}" for cls in clf.classes_])
prob_df["prediccion"] = y_pred
prob_df["real"] = y_test.values
prob_df["index_original"] = index_test.values

# 3.2 Agregar columnas originales para análisis
original_test_data = df_sample.loc[prob_df["index_original"].values]
resultados_finales = pd.concat([original_test_data.reset_index(drop=True), prob_df.reset_index(drop=True)], axis=1)

# 3.3 Guardar CSV
resultados_finales.to_csv("predicciones_random_forest.csv", index=False)


c:\Users\nicol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nicol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nicol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     79845
           1       0.67      0.70      0.68     15718
           2       0.64      0.71      0.67     29611
           3       0.56      0.54      0.55     24417
           4       0.59      0.57      0.58     20907
           5       0.53      0.50      0.51     12116
           6       0.54      0.48      0.51      6410
           7       0.54      0.48      0.51      2712
           8       0.54      0.36      0.43       710
           9       0.57      0.32      0.41       135
          10       0.00      0.00      0.00         5

    accuracy                           0.76    192586
   macro avg       0.56      0.51      0.53    192586
weighted avg       0.76      0.76      0.76    192586



In [18]:
resultados_finales.head()

,FECHA_COTIZACION,NUMERO_SOLICITUD,NUMERO_IDENTIFICACION,NUMERO_SECUENCIA,TIPO_RENTA,MESES_DIFERIDOS,MODALIDAD_RENTA,MESES_GARANTIZADOS,MESES_AUMENTO_TEMPORAL,PORCENTAJE_AUMENTO_TEMPORAL,...,proba_pos_4,proba_pos_5,proba_pos_6,proba_pos_7,proba_pos_8,proba_pos_9,proba_pos_10,prediccion,real,index_original
0,2024-06-13,132262802,195825026,1,I,0,G,240,84,100,...,0.57,0.07,0.01,0.0,0.01,0.0,0.0,4,3,480659
1,2024-12-02,137464902,201885839,2,D,60,S,0,0,0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0,0,898908
2,2024-04-29,130793601,194128904,1,I,0,S,0,0,0,...,0.04,0.04,0.18,0.0,0.00,0.0,0.0,2,2,365176
3,2024-12-31,138286601,202793343,1,I,0,S,0,36,100,...,0.10,0.01,0.01,0.0,0.00,0.0,0.0,3,3,960341
4,2024-08-30,134861201,198568233,1,I,0,G,180,0,0,...,0.17,0.00,0.00,0.0,0.00,0.0,0.0,3,5,672365
